In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv("train_V2.csv")
train.columns = ['Id','groupId','matchId','assists','boosts','damageDealt','DBNOs','headshotKills','heals','killPlace','killPoints','kills','killStreaks','longestKill','matchDuration','matchType','maxPlace','numGroups','rankPoints','revives','rideDistance','roadKills','swimDistance','teamKills','vehicleDestroys','walkDistance','weaponsAcquired','winPoints','winPlacePerc']
#train = train_complete.sample(500000)

In [3]:
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
train['healsandboosts'] = train['heals'] + train['boosts']
train['totaldistance'] = train['rideDistance'] + train['swimDistance'] + train['walkDistance']
train.describe()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,playersJoined,healsandboosts,totaldistance
count,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,...,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446965e+06,4.446966e+06,4.446966e+06,4.446966e+06
mean,2.338149e-01,1.106908e+00,1.307171e+02,6.578755e-01,2.268196e-01,1.370147e+00,4.759935e+01,5.050060e+02,9.247833e-01,5.439551e-01,...,4.509322e+00,2.386841e-02,7.918208e-03,1.154218e+03,3.660488e+00,6.064601e+02,4.728216e-01,9.419479e+01,2.477055e+00,1.764843e+03
std,5.885731e-01,1.715794e+00,1.707806e+02,1.145743e+00,6.021553e-01,2.679982e+00,2.746294e+01,6.275049e+02,1.558445e+00,7.109721e-01,...,3.050220e+01,1.673935e-01,9.261157e-02,1.183497e+03,2.456544e+00,7.397004e+02,3.074050e-01,6.686538e+00,3.879985e+00,2.183117e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.551000e+02,2.000000e+00,0.000000e+00,2.000000e-01,9.300000e+01,0.000000e+00,1.581000e+02
50%,0.000000e+00,0.000000e+00,8.424000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,6.856000e+02,3.000000e+00,0.000000e+00,4.583000e-01,9.500000e+01,1.000000e+00,7.882000e+02
75%,0.000000e+00,2.000000e+00,1.860000e+02,1.000000e+00,0.000000e+00,2.000000e+00,7.100000e+01,1.172000e+03,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.976000e+03,5.000000e+00,1.495000e+03,7.407000e-01,9.700000e+01,4.000000e+00,2.728447e+03
max,2.200000e+01,3.300000e+01,6.616000e+03,5.300000e+01,6.400000e+01,8.000000e+01,1.010000e+02,2.170000e+03,7.200000e+01,2.000000e+01,...,3.823000e+03,1.200000e+01,5.000000e+00,2.578000e+04,2.360000e+02,2.013000e+03,1.000000e+00,1.000000e+02,8.100000e+01,4.127010e+04


In [4]:
train['killsNorm'] = train['kills']*((100-train['playersJoined'])/100 + 1)
train['damageDealtNorm'] = train['damageDealt']*((100-train['playersJoined'])/100 + 1)
train['maxPlaceNorm'] = train['maxPlace']*((100-train['playersJoined'])/100 + 1)
train['matchDurationNorm'] = train['matchDuration']*((100-train['playersJoined'])/100+1)
train['killPlace'] = train['killPlace']*((100 - train['playersJoined'])/100 + 1)

In [5]:
train['killswithoutmoving'] = ((train['totaldistance']==0) & (train['kills'] > 0))


In [6]:
print(train[train['killswithoutmoving'] == True].shape)
train.drop(train[train['killswithoutmoving'] == True].index, inplace=True)
print(train[train['kills'] > 40].shape)
print(train[(train['headshotKills']/train['kills'] == 1) & (train['kills'] > 12)].shape)

print(train[(train['totaldistance'] == train['swimDistance'])&(train['winPlacePerc'] > 0.80)].shape)
train.drop(train[(train['totaldistance'] == train['swimDistance'])&(train['winPlacePerc'] > 0.80)].index, inplace=True)
train.drop(train[train['kills'] > 40].index, inplace=True)
train.drop(train[(train['headshotKills']/train['kills'] == 1) & (train['kills'] > 12)].index, inplace=True)

(1535, 37)
(32, 37)
(6, 37)
(991, 37)


In [7]:
print(train[train['weaponsAcquired'] > 50].shape)
train.drop(train[train['weaponsAcquired'] > 50].index, inplace=True)
print(train[train['heals'] > 35].shape)
train.drop(train[train['heals'] > 35].index, inplace=True)



(133, 37)
(273, 37)


In [8]:
# ONLY  FOR THE ENTIRE DATASET ###################
train[train['winPlacePerc'].isnull()]
train.drop(2744604, inplace=True)

In [9]:
print(train[(train['weaponsAcquired'] > 35) & (train['totaldistance'] == 0)].shape)
train.drop(train[(train['weaponsAcquired'] > 35) & (train['totaldistance'] == 0)].index, inplace=True)

(1, 37)


In [10]:
print(train[(train['weaponsAcquired'] > 30) & (train['totaldistance'] < 50) & (train.kills > 30)].shape)
train.drop(train[(train['weaponsAcquired'] > 30) & (train['totaldistance'] < 50) & (train.kills > 30)].index, inplace=True)

(2, 37)


In [11]:
train[train.longestKill > 950].shape
train.drop(train[train['longestKill'] > 950].index, inplace=True)

In [20]:
#columns = ['killsNorm', 'damageDealtNorm', 'matchDurationNorm', 'assists', 'healsandboosts', 'DBNOs', 'killPlace', 'walkDistance', 'winPoints'
#          ,'weaponsAcquired', 'killStreaks']
columns = ['killsNorm', 'damageDealtNorm', 'matchDurationNorm','healsandboosts', 'DBNOs', 'killPlace','walkDistance', 'winPoints'
          ,'weaponsAcquired', 'killStreaks', 'longestKill', 'teamKills', 'maxPlace', 'assists', 'revives'
          ,'numGroups']

0.12652996538579137
0.8304411406791566


In [33]:
PCA_X = train.loc[:, columns].values
PCA_y = train.loc[:, ['winPlacePerc']].values

PCA_X = StandardScaler().fit_transform(PCA_X)

pca = PCA(n_components=8)
principal_components = pca.fit_transform(PCA_X)
principal_df = pd.DataFrame(data = principal_components, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5', '  principal component 6', 'principal component 7', 'principal component 8'])

pca_df = pd.concat([principal_df, train[['winPlacePerc']]], axis = 1)

train_X, val_X, train_y, val_y = train_test_split(PCA_X,PCA_y, test_size=0.2, random_state=1)
model = LinearRegression()
model.fit(train_X, train_y)
pred = model.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, pred)))
print(model.score(val_X, val_y))

0.12652996538579137
0.8304411406791566


In [ ]:
train_X, val_X, train_y, val_y = train_test_split(PCA_X,PCA_y, test_size=0.2, random_state=1)
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0, min_samples_leaf=3, max_features='sqrt')
regressor.fit(train_X, train_y)
predict = regressor.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, predict)))
print(regressor.score(val_X, val_y))

In [ ]:
X = train[columns]
y = train['winPlacePerc']

train_X, val_X, train_y, val_y = train_test_split(X,y, test_size=0.2, random_state=1)
model = LinearRegression()
model.fit(train_X, train_y)
pred = model.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, pred)))
print(model.score(val_X, val_y))

In [21]:
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0, min_samples_leaf=3, max_features='sqrt')
regressor.fit(train_X, train_y)
predict = regressor.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, predict)))
print(regressor.score(val_X, val_y))

0.0808319208667455
0.9308010497031849


In [14]:
train['Solo'] = (train['numGroups'] > 50).astype(np.int8)
train['Duo'] = (train['numGroups'] <= 50 & train['numGroups']).astype(np.int8)
train['Squad'] = (train['numGroups']).astype(np.int8)

In [15]:
train.Duo.describe()

count    4.443908e+06
mean     6.105482e-02
std      2.394309e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: Duo, dtype: float64